# Práctica: Los Juegos del Hambre

ESTUDIANTE: Javier Herraez, Pablo Peman


<table><tr>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/breakfast.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/hamburger.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/fruits.jpg" style="width:300px;height:300px;"></td>
</tr></table>

En esta práctica vamos a enfrentarnos a un problema desafiante de clasificación de imágenes, construyendo una red neuronal profunda que sea capaz de clasificar entre diferentes tipos de comida. ¡Que comiencen los Juegos del Hambre!

## Guidelines

A lo largo del notebook encontrarás celdas que debes rellenar con tu propio código. Sigue las instrucciones del notebook y presta atención a los siguientes iconos:

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Deberás resolver el ejercicio escribiendo tu propio código o respuesta en la celda inmediatamente inferior.</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
Esto es una pista u observación de utilidad que puede ayudarte a resolver el ejercicio. Presta atención a estas pistas para comprender el ejercicio en mayor profundidad.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
Este es un ejercicio avanzado que te puede ayudar a profundizar en el tema, y a conseguir una calificación más alta. ¡Buena suerte!</font>

***

Para evitar problemas con imports o incompatibilidades se recomienda ejecutar este notebook en uno de los [entornos de Deep Learning recomendados](https://github.com/albarji/teaching-environments-deeplearning), o hacer uso [Google Colaboratory](https://colab.research.google.com/). Si usas Colaboratory, asegúrate de [conectar una GPU](https://colab.research.google.com/notebooks/gpu.ipynb), y de haber [deactivado otras sesiones que tuvieras activas](https://stackoverflow.com/a/53441194/2436578).

El siguiente código mostrará todas las gráficas en el propio notebook en lugar de generar una nueva ventana.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Finalmente, si necesitas ayuda en el uso de cualquier función Python, coloca el cursor sobre su nombre y presiona Shift+Tab. Aparecerá una ventana con su documentación. Esto solo funciona dentro de celdas de código.

¡Vamos alla!

## Obtención de datos

Vamos a usar un dataset de imágenes de comida disponible en [Kaggle](https://www.kaggle.com/trolukovich/food11-image-dataset). Para descargarlo, necesitarás crear una cuenta de usuario en Kaggle, y obtener tus credenciales de la API. Puedes hacerlo siguiendo las instrucciones de [esta sección](https://github.com/Kaggle/kaggle-api#api-credentials). ¡Ojo! Tus credenciales de la API no son lo mismo que la contraseña que utilizas para acceder a tu cuenta en Kaggle.

Una vez tengas el fichero JSON con tus credenciales, puedes declararlas en este notebook asignando las variables de entorno adecuadas, de la siguiente manera

    import os

    os.environ["KAGGLE_USERNAME"] = "YOUR KAGGLE USERNAME HERE"
    os.environ["KAGGLE_KEY"] = "YOUR KAGGLE KEY HERE"
    
Cuando lo hayas hecho, podrás descargar el dataset a la máquina donde esté corriendo este notebook usando el siguiente comando

    !kaggle datasets download trolukovich/food11-image-dataset --unzip -p YOUR_LOCAL_FOLDER
    
donde debes indicar el nombre de un directorio válido como "YOUR_LOCAL_FOLDER". Si prefieres descargar los datos en la misma carpeta que este notebook, puedes quitar la parte `-p YOUR_LOCAL_FOLDER` del comando.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Crea tu propia cuenta de Kaggle (si no tienes ya una), obtén tus credenciales, y usa la celda inferior para declarar tu nombre de usuario de Kaggle y tu clave como variables de entorno. A continuación, usa la misma celda para descargar el dataset de imágenes.
    
¡Ojo! Debes mantener estas credenciales en secreto, ya que son personales a tu usuario de Kaggle. Recuerda borrarlas de la celda antes de entregar este notebook.
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

import os

os.environ["KAGGLE_USERNAME"] = "pablopeman"
os.environ["KAGGLE_KEY"] = "86a13bd99e3707adb88f667956d2414f"

!kaggle datasets download trolukovich/food11-image-dataset --unzip


100% 1.08G/1.08G [00:05<00:00, 233MB/s]
100% 1.08G/1.08G [00:05<00:00, 215MB/s]


Revisa ahora la carpeta en la que has descargado los datos. Verás que contiene 3 subdirectorios:

* **training**, contiene las imágenes a utilizar para entrenar el modelo.
* **validation**, contiene imágenes adicionales que podrías usar como datos de entrenamiento adicionales, o para algún tipo de estrategia de validación como Early Stopping.
* **evaluation**, contiene las imágenes que debes utilizar para testear el modelo. Las imágenes de esta carpeta **solo** pueden utilizarse para medir el rendimiento del modelo tras su entrenamiento, y para nada más.

Además de esto, dentro de cada una de estas carpetas encontrarás una subcarpeta para cada una de las 11 clases de comida:

* Bread (panes)
* Dairy product (lácteos)
* Dessert (postres)
* Egg (huevos)
* Fried food (fritos)
* Meat (carnes)
* Noodles-Pasta (pasta)
* Rice (arroz)
* Seafood (pescado y marisco)
* Soup (sopas)
* Vegetable-Fruit (vegetales y frutas)

Esta es una forma estándar de organizar los datasets de imágenes: una carpeta para cada clase. Para facilitar los pasos de procesamiento que vendrán a continuación, vamos a definir algunas variables que nos indiquen dónde están almacenados los diferentes conjuntos de datos.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Crea variables <b>TRAINDIR</b>, <b>VALDIR</b> y <b>TESTDIR</b>, cada una conteniendo una cadena de texto con la ruta al directorio donde están los datos de entrenamiento, validación y evaluación, respectivamente.
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE
TRAINDIR = "/content/training"
VALDIR= "/content/validation"
TESTDIR= "/content/evaluation"

### Reducción de clases

Con el fin de hacer este problema más accesible de cara a la práctica, vamos a centrarnos solo en seis de las clases de comida disponibles: `Bread`, `Dairy product`, `Dessert`, `Egg`, `Fried food` y `Meat`. Para ello, se provee el código siguiente, que elimina de los datos descargados las carpetas correspondientes a imágenes de las otras clases.

In [ ]:
from glob import glob
import os

valid_classes = {"Bread", "Dairy product", "Dessert", "Egg", "Fried food", "Meat"}
datasets = {TRAINDIR, VALDIR, TESTDIR}

for dataset in datasets:
    for classdir in glob(f"{dataset}/*"):  # Find subfolders with classes
        if classdir.split("/")[-1] not in valid_classes:  # Ignore those in valid_classes
            print(f"Deleting {classdir}...")
            for fname in glob(f"{classdir}/*.jpg"):  # Remove each image file
                os.remove(fname)
            os.rmdir(classdir)  # Remove folder

Deleting /content/training/Soup...
Deleting /content/training/Seafood...
Deleting /content/training/Vegetable-Fruit...
Deleting /content/training/Noodles-Pasta...
Deleting /content/training/Rice...
Deleting /content/validation/Soup...
Deleting /content/validation/Seafood...
Deleting /content/validation/Vegetable-Fruit...
Deleting /content/validation/Noodles-Pasta...
Deleting /content/validation/Rice...
Deleting /content/evaluation/Soup...
Deleting /content/evaluation/Seafood...
Deleting /content/evaluation/Vegetable-Fruit...
Deleting /content/evaluation/Noodles-Pasta...
Deleting /content/evaluation/Rice...


## Procesando imágenes desde ficheros

Este dataset de imágenes es grande, con imágenes de mayor resolución que las que hemos utilizado en el tutorial del MNIST, y cada una de ellas teniendo diferentes tamaños y relación de aspecto. Además, mientras que para el MNIST teníamos una función de keras que preparaba los datos para nosotros, en esta ocasión tendremos que realizar el trabajo de carga y procesamiento de las imágenes.

Una forma conveniente de hacer todo este trabajo es a través de la función Keras `image_dataset_from_directory`. Esta función crea un objeto `Dataset` de TensorFlow con todas las imágenes de un directorio, cargándolas en memoria de forma dinámica solo cuando la red neuronal necesita utilizarlas. Esta función también nos permite especificar algunas opciones de preprocesamiento muy útiles.

Por ejemplo, podemos crear un `Dataset` con los datos en la carpeta de training:

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

image_size = 32
batch_size = 64

train_dataset = image_dataset_from_directory(
    TRAINDIR, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = 'categorical'
)

Found 6082 files belonging to 6 classes.


Observa los parámetros que se han utilizado para configurar el dataset:

* El **directorio** desde el que cargar las imágenes.
* Un **tamaño de imagen (image_size)** que se utilizará para redimensionar todas las imágenes cargadas a ese tamaño común, en este caso 32x32 píxeles.
* El **tamaños de los lotes (batch_size)** de imágenes a ser generados. Nótese que definimos aquí este parametro en lugar de en el paso `fit` de la red, como hemos hecho en otros ejercicios, porque el objeto `Dataset` resultante hará uso de esta información para mantener en memoria solo algunos batches de imágenes, ahorrando así memoria.
* El **modo de etiquetado (label_mode)**, esto es, la codificación de las etiquetas a utilizar. `categorical` significa que utilizaremos la ya conocida codificación one-hot.

Un objeto `Dataset` funciona de manera muy similar a un generador de Python, lo que significa que podemos iterar sobre él para obtener batches de imágenes ya preprocesadas. Por ejemplo, el siguiente código inicia un bucle para extraer todos los batches del `Dataset`, nos muestra el contenido el primero, y detiene la iteración.

In [ ]:
for X_batch, y_batch in train_dataset:
    print(f"Shape of input batch: {X_batch.shape}")
    print(f"Shape of output batch: {y_batch.shape}")
    print(f"Input batch:\n{X_batch}")
    print(f"Output batch:\n{y_batch}")
    break

Shape of input batch: (64, 32, 32, 3)
Shape of output batch: (64, 6)
Input batch:
[[[[5.05000000e+01 6.45000000e+01 3.50000000e+01]
   [1.18750000e+02 9.12500000e+01 3.70000000e+01]
   [1.59750000e+02 1.29750000e+02 7.45000000e+01]
   ...
   [7.70000000e+01 5.35000000e+01 3.30000000e+01]
   [6.95000000e+01 4.75000000e+01 2.50000000e+01]
   [6.75000000e+01 4.10000000e+01 2.30000000e+01]]

  [[4.70000000e+01 4.15000000e+01 1.40000000e+01]
   [8.77500000e+01 7.37500000e+01 3.22500000e+01]
   [1.36000000e+02 1.11750000e+02 6.47500000e+01]
   ...
   [6.80000000e+01 4.60000000e+01 2.27500000e+01]
   [6.37500000e+01 4.30000000e+01 1.77500000e+01]
   [5.92500000e+01 3.85000000e+01 2.05000000e+01]]

  [[3.35000000e+01 3.47500000e+01 1.60000000e+01]
   [6.42500000e+01 5.62500000e+01 2.62500000e+01]
   [1.19750000e+02 9.62500000e+01 5.22500000e+01]
   ...
   [6.65000000e+01 4.30000000e+01 1.85000000e+01]
   [6.62500000e+01 4.75000000e+01 2.27500000e+01]
   [1.17250000e+02 9.40000000e+01 5.2750000

Podemos ver que, efectivamente, el generador produce un tensor de datos de entrada de las dimensiones apropiadas para poder introducirlo en la red neuronal, y que las salidas también se han codificado correctamente como one-hot.
No obstante, todavía hay un problema con los datos: los valores de los píxeles están en el rango [0, 255], lo cual puede producir problemas de entrenamiento. Resolveremos este punto después, en la definición de la red neuronal, mediante una capa especial. Por ahora vamos a cotinuar, definiendo una función que construya los `Dataset` para los datos de entrenamiento, validación y test.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Crea una función <b>create_datasets</b> que reciba los siguiente parámetros:
    <ul>
      <li><b>traindir</b>: el directorio donde están localizadas las imágenes de entrenamiento.</li>
      <li><b>valdir</b>: el directorio donde están localizadas las imágenes de validación.</li>
      <li><b>testdir</b>: el directorio donde están localizadas las imágenes de test.</li>
      <li><b>image_size</b>: el tamaño que se utilizará para redimensionar todas las imágenes a una resolución común.</li>
      <li><b>batch_size</b>: el tamaño de los batches de imágenes que serán generados.</li>
    </ul>
    La función debe crear objetos `Dataset` para los directorios de entrenamiento, validación y test, y devolver los tres datasets creados como

    return train_dataset, val_dataset, test_dataset
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

def create_datasets (traindir,valdir,testdir,image_size,batch_size):
  train_dataset=image_dataset_from_directory(
    traindir, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = 'categorical'
    )
  val_dataset=image_dataset_from_directory(
    valdir, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = 'categorical'
    )
  test_dataset=image_dataset_from_directory(
    testdir, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = 'categorical'
    )
  return train_dataset, val_dataset, test_dataset

  

Probemos que la función que has implementado funciona correctamente

In [ ]:
import tensorflow as tf

train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=32, batch_size=64)

# Test whether all returned objects are valid Tensorflow datasets
assert isinstance(train_dataset, tf.data.Dataset)
assert isinstance(val_dataset, tf.data.Dataset)
assert isinstance(test_dataset, tf.data.Dataset)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


¡Ahora que tenemos nuestros `Dataset` podemos entrenar una red profunda con ellos! Como ejemplo, vamos a construir una red convolucional extremadamente simple. Nótese cómo hemos añadido una capa especial de preprocesado llamada `Rescaling`, que será la encargada de normalizar los valores de los píxeles al rango [0, 1] cada vez que la red reciba una imagen.

¡Ojo! Esta red tan simple no producirá errores al ejecutar, pero tiene algunos fallos de diseño que deberás corregir cuando crees tu propia red, más adelante en esta práctica.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Convolution2D, Rescaling

model = Sequential()
model.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model.add(Convolution2D(4, 3, activation='linear'))
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])

El método `fit` de un modelo Keras puede recibir un objeto `Dataset` como datos de entrenamiento, en lugar de un par de tensores (entradas, salidas). Como al construir los `Dataset` ya especificamos el tamaño de batch, no es necesario indicarlo ahora.

In [ ]:
model.fit(train_dataset, epochs=1)

96/96 [==============================] - 13s 110ms/step - loss: 1.6872 - accuracy: 0.2738


Análogamente, podemos evaluar el rendimiento de nuestro modelo sobre el `Dataset` de test de la siguiente manera

In [ ]:
loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 5s 109ms/step - loss: 1.8847 - accuracy: 0.2449
Loss 1.88, accuracy 24.5%


Este nivel de acierto puede parecer pobre, pero ten en cuenta que hemos usado un modelo muy simple y que el problema es de 6 clases. ¿Serás capaz de hacerlo mejor?

## Construyendo tu propia red

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Diseña una red neuronal profunda que obtenga el mejor acierto posible sobre los datos de test. Puedes usar los datos de entrenamiento y validación como te parezca mejor, pero <b>sólo</b> puedes usar los datos de test para evaluar el acierto de tu modelo. Debes obtener una red capaz de alcanzar al menos un 40% de acierto sobre los datos de test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Algunas recomendaciones y estrategias que pueden ayudar a mejorar tu diseño de red:

    
- Usa todos los trucos que has aprendido en los ejercicios anteriores: capas convolucionales + pooling, activaciones ReLU, dropout... asegúrate también de utilizar un buen optimizador, con una función de error (loss) adecuada, así como una función de activación en la capa de salida que sea adecuada para esta clase de problema (clasificación multiclase).
- Empieza por redes pequeñas, con un número pequeño de parámetros, de forma que puedas comprobar rápidamente qué tal funcionan. Después, puedes hacer tu red más grande en tres direcciones: mayor tamaño de imágenes de entrada, más capas, y más kernels por capa convolucional o unidades por capa densa. Si aumentas el tamaño de las imágenes de entrada, asegúrate de añadir también más capas Convolution+Pooling, para que así a la capa Flatten solo lleguen imágenes muy pequeñas (10x10 píxeles o menos).
- Configurar los `Dataset` para que carguen imágenes de mayor tamaño puede mejorar significativamente el rendimiento de tu red. Pero ten cuidado, ¡también puedes encontrarte errores de falta de memoria (CUDA memory error) si cargas imágenes a un tamaño demasiado grande! Para esta práctica, un tamaño mayor a 256 puede ser demasiado grande...
- Si observas grandes diferencias de loss entre los datos de entrenamiento y validación o test, prueba a incrementar el nivel de Dropout en las capas Dense.
- ¡Usa los datos de validación! Por ejemplo, usa una <a href="https://keras.io/api/callbacks/early_stopping/">**estrategia de EarlyStopping**</a> para monitorizar el loss de los datos de validación, y así detener el entrenamiento cuando tras un número de épocas esa loss no haya decrecido. Configurar la EarlyStopping para restaurar los mejores parámetros encontrados durante la optimización también puede resultarte útil.
- Si tu red obtiene resultados muy buenos, del orden del 90% o más de acierto en test... sospecha. Es posible que estés mezclando los datos de entrenamiento y test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
    
Como ejercicio avanzado, añade las siguiente estrategias a tu red:

- Usa **técnicas de "image augmentation"** para aumentar artificialmente tu dataset de entrenamiento. Para ello, explora las <a href="https://keras.io/api/layers/preprocessing_layers/image_augmentation/">capas de augmentation disponibles en Keras</a>.
- Usa capas de <a href="https://keras.io/api/layers/normalization_layers/batch_normalization/">BatchNormalization</a> para facilitar el entrenamiento de la red.
    
Usando estos trucos y los mencionados en el punto anterior, es posible obtener más de un 60% de acierto en el conjunto de test.

</font>

***

In [ ]:
##### INSERT YOUR CODE HERE

from tensorflow.keras.layers.experimental.preprocessing import RandomFlip,RandomRotation,RandomZoom,RandomTranslation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization, MaxPooling2D

train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=128, batch_size=64)

img_rows = 128
img_cols = 128


kernel_size = 3
pool_size = 2 

custom_early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=8, 
    min_delta=0.001
)

model_1 = Sequential([
              RandomFlip('horizontal'),
              RandomRotation(0.2),
              RandomZoom(0.3,0.2),
              BatchNormalization()
             ])

model_1.add(Convolution2D(32, (kernel_size, kernel_size), input_shape=(img_rows, img_cols, 3), activation="relu"))
model_1.add(MaxPooling2D(pool_size, strides=2))
model_1.add(Convolution2D(32, (kernel_size,kernel_size), activation="relu"))
model_1.add(MaxPooling2D(pool_size, strides=2))
model_1.add(Convolution2D(32, (kernel_size,kernel_size), activation="relu"))
model_1.add(MaxPooling2D(pool_size, strides=2))

model_1.add(Flatten())
model_1.add(Dense(256, activation="relu"))
model_1.add(Dropout(0.5))
model_1.add(Dense(32, activation="relu"))
model_1.add(Dense(6, activation="softmax"))

model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_1.fit(
    train_dataset,
    epochs=30,
    verbose=1 ,
    callbacks=[custom_early_stopping],
    validation_data=val_dataset
)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.
Epoch 1/30
96/96 [==============================] - 21s 185ms/step - loss: 1.6144 - accuracy: 0.3234 - val_loss: 1.6033 - val_accuracy: 0.3074
Epoch 2/30
96/96 [==============================] - 18s 185ms/step - loss: 1.5376 - accuracy: 0.3560 - val_loss: 1.4716 - val_accuracy: 0.4146
Epoch 3/30
96/96 [==============================] - 17s 170ms/step - loss: 1.4787 - accuracy: 0.3959 - val_loss: 1.3828 - val_accuracy: 0.4417
Epoch 4/30
96/96 [==============================] - 20s 199ms/step - loss: 1.4426 - accuracy: 0.4101 - val_loss: 1.5166 - val_accuracy: 0.4065
Epoch 5/30
96/96 [==============================] - 18s 182ms/step - loss: 1.4321 - accuracy: 0.4211 - val_loss: 1.3958 - val_accuracy: 0.4426
Epoch 6/30
96/96 [==============================] - 19s 192ms/step - loss: 1.4357 - accuracy: 0.4107 - val_loss: 1.4272 - val_accuracy: 0.4483
Epoch 7/30
96/96 [=

In [ ]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 128, 128, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 128, 128, 3)      0         
 tion)                                                           
                                                                 
 random_zoom (RandomZoom)    (None, 128, 128, 3)       0         
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 3)      12        
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)      

In [ ]:
score = model_1.evaluate(test_dataset)
print("Test loss", score[0])
print("Test accuracy", score[1])

33/33 [==============================] - 5s 122ms/step - loss: 1.3068 - accuracy: 0.5647
Test loss 1.3068479299545288
Test accuracy 0.5647342801094055


In [ ]:
train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=128, batch_size=64)

img_rows = 128
img_cols = 128
kernel_size = 3
pool_size = 2 

custom_early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=8, 
    min_delta=0.001
)

model_2 = Sequential([
              RandomFlip('horizontal'),
              RandomRotation(0.2),
              RandomZoom(0.3,0.2),
              BatchNormalization()
             ])

model_2.add(Convolution2D(64, (kernel_size, kernel_size), input_shape=(img_rows, img_cols, 3), activation="relu"))
model_2.add(MaxPooling2D(pool_size, strides=2))
model_2.add(Convolution2D(32, (kernel_size, kernel_size), activation="relu"))
model_2.add(MaxPooling2D(pool_size, strides=2))
model_2.add(Convolution2D(32, (kernel_size, kernel_size), activation="relu"))
model_2.add(MaxPooling2D(pool_size, strides=2))

model_2.add(Flatten())
model_2.add(Dense(256, activation="relu"))
model_2.add(Dropout(0.5))
model_2.add(Dense(128, activation="relu"))
model_2.add(Dropout(0.5))
model_2.add(Dense(32, activation="relu"))
model_2.add(Dense(6, activation="softmax"))

model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model_2.fit(
    train_dataset,
    epochs=30,
    verbose=1 ,
    callbacks=[custom_early_stopping],
    validation_data=val_dataset
)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.
Epoch 1/30
96/96 [==============================] - 20s 181ms/step - loss: 1.6898 - accuracy: 0.2882 - val_loss: 1.6298 - val_accuracy: 0.3126
Epoch 2/30
96/96 [==============================] - 18s 174ms/step - loss: 1.5850 - accuracy: 0.3298 - val_loss: 1.4783 - val_accuracy: 0.3933
Epoch 3/30
96/96 [==============================] - 18s 175ms/step - loss: 1.5300 - accuracy: 0.3573 - val_loss: 1.4633 - val_accuracy: 0.4023
Epoch 4/30
96/96 [==============================] - 18s 174ms/step - loss: 1.5034 - accuracy: 0.3696 - val_loss: 1.5283 - val_accuracy: 0.3819
Epoch 5/30
96/96 [==============================] - 18s 178ms/step - loss: 1.5019 - accuracy: 0.3752 - val_loss: 1.4223 - val_accuracy: 0.4241
Epoch 6/30
96/96 [==============================] - 18s 180ms/step - loss: 1.4546 - accuracy: 0.4000 - val_loss: 1.3784 - val_accuracy: 0.4369
Epoch 7/30
96/96 [=

In [ ]:
model_2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_1 (RandomFlip)  (None, 128, 128, 3)       0         
                                                                 
 random_rotation_1 (RandomRo  (None, 128, 128, 3)      0         
 tation)                                                         
                                                                 
 random_zoom_1 (RandomZoom)  (None, 128, 128, 3)       0         
                                                                 
 batch_normalization_1 (Batc  (None, 128, 128, 3)      12        
 hNormalization)                                                 
                                                                 
 conv2d_4 (Conv2D)           (None, 126, 126, 64)      1792      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 63, 63, 64)      

In [ ]:
score = model_2.evaluate(test_dataset)
print("Test loss", score[0])
print("Test accuracy", score[1])

33/33 [==============================] - 5s 124ms/step - loss: 1.1810 - accuracy: 0.5478
Test loss 1.1809520721435547
Test accuracy 0.5478261113166809


## Transfer learning

Aunque diseñar nuestra propia red puede producir algunos resultados aceptables, suele ser mejor aprovechar el conocimiento ya existente en una red pre-entrenada. Esto no solo nos lleva a resultados mejores, sino que además nos ahorra mucho tiempo de diseño de la red. Para ello, el módulo [Keras Applications](https://keras.io/api/applications/) contiene varios diseños de redes listos para su uso. Por ejemplo, para hacer uso de la famosa red VGG16 hacemos lo siguiente

In [ ]:
from tensorflow.keras.applications import VGG16
train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=32, batch_size=64)

vgg16_model = VGG16(include_top=False, input_shape=(image_size, image_size, 3))

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.


Por defecto, todas las redes de Keras Applications están precargadas con los pesos que se obtuvieron al entrenar la red sobre el dataset de la competición de [ImageNet](http://www.image-net.org/). Para adaptar la red a nuestro problema, hemos necesitado especificar la resolución de nuestras imágenes (`input_shape`), así como eliminar las capas de salida (`top`) de la red original, dado que nosotros tendremos un número diferente de clases.

Ahora, ¿cómo hacemos para transferir el aprendizaje de esta red? Vamos a ver cómo implementar la estrategia de "bottleneck features". En primer lugar, marcamos el modelo VGG16 como no entrenable, para que sus parámetros se mantengan congelados.

In [ ]:
vgg16_model.trainable = False

Hecho esto, vamos a construir una red neuronal que incluya la VGG16 como una de sus "capas". Es necesario tener en cuenta que la red VGG16 se entrenó realizando una normalización muy específica de las imágenes de entrenamiento, y nosotros debemos seguir ese mismo proceso para que la red se comporte correctamente. Convenientemente, Keras también nos da una funcionalidad para replicar la normalización que la VGG16 necesita.

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

Podemos probar esta normalización con alguna de las imágenes de nuestro dataset

In [ ]:
for X_batch, _ in train_dataset:
    break
    
print(f"Before normalizing: {X_batch[0, :3, :3, :]}")
print(f"After normalizing: {preprocess_input(X_batch)[0, :3, :3, :]}")

Before normalizing: [[[69.390625 46.890625 46.390625]
  [58.984375 35.203125 37.796875]
  [51.53125  28.5      30.84375 ]]

 [[53.4375   32.734375 32.4375  ]
  [56.203125 34.703125 35.203125]
  [59.515625 38.015625 38.515625]]

 [[53.9375   32.9375   31.9375  ]
  [54.90625  32.40625  31.90625 ]
  [60.       37.5      37.      ]]]
After normalizing: [[[-57.548378 -69.888374 -54.289375]
  [-66.14213  -81.575874 -64.695625]
  [-73.09525  -88.279    -72.14875 ]]

 [[-71.5015   -84.044624 -70.2425  ]
  [-68.73588  -82.075874 -67.476875]
  [-65.42338  -78.763374 -64.164375]]

 [[-72.0015   -83.8415   -69.7425  ]
  [-72.03275  -84.37275  -68.77375 ]
  [-66.939    -79.279    -63.68    ]]]


La normalización realizada por la VGG16 consiste en invertir el orden de los canales de color (RGB -> BGR), y restar los valores medios sobre todo el dataset ImageNet para cada canal de color por separado. Afortunadamente, la función `preprocess_input` que hemos importado hace todo este trabajo por nosotros. Además, podemos incrustar esta función como la primera capa de nuestra red, cumpliendo el papel de la capa `Rescaling` que utilizamos en el apartado anterior. Esto es posible gracias a la capa `Lambda`, que permite construir una capa Keras en base a cualquier función de Tensorflow. De modo que, vamos a comenzar nuestro diseño de red con esta capa.

In [ ]:
from tensorflow.keras.layers import Lambda

model = Sequential()
model.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

Tras esto, podemos añadir toda la red VGG16 como si fuera una nueva capa, y nuestras propias capas después de ella. A continuación tenemos un ejemplo de esta clase de diseño, aunque es importante destacar que es un diseño muy sencillo que contiene algunos fallos; una red real para hacer transfer learning debería tener un diseño mejor.

In [ ]:
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 32, 32, 3)         0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 6)                 3078      
                                                                 
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


Observa cómo en el resumen del modelo podemos ver que la red completa tiene millones de parámetros, pero dado que hemos congelado toda la parte de la red perteneciente a la VGG16, solo unos pocos miles de parámetros son entrenables (trainable): aquellos correspondientes a la capa Dense que hemos colocado al final.

Ya podemos compilar y entrenar el modelo a la manera habitual.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])
model.fit(train_dataset, epochs=1)

loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

33/33 [==============================] - 4s 92ms/step - loss: 205.5580 - accuracy: 0.2725
Loss 2.06e+02, accuracy 27.2%


<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Usando la estrategia "bottleneck" presentada, implementa una red que haga transfer learning desde la red VGG16, con un diseño correcto. Si lo haces adecuadamente, esta red debe obtener mejores resultados que con la red que diseñaste en el apartado anterior, y con al menos un 80% de acierto sobre el conjunto de test.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Algunos consejos para mejorar tu diseño de red:
    
- Incluye una o más capas Dense, con sus funciones de activación apropiadas, antes de la capa de salida.
- Intenta usar una capa de tipo [GlobalAveragePooling](https://keras.io/api/layers/pooling_layers/global_average_pooling2d/) en lugar de la capa Flatten. Esta capa calcula una media de todos los valores de píxeles para cada canal, y en algunas ocasiones produce mejores resultados que la capa Flatten.
- ¡Y no olvides todos los consejos del apartado anterior! También aplican aquí.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>

Para mejorar aún más los resultados de tu red, utiliza las siguientes ideas:

- Usa las estrategias PRO del ejercicio anterio.
- Prueba otras redes pre-entrenadas de <a href="https://keras.io/api/applications/">Keras Applications</a>, como ResNet, Xception o EfficientNet.
- Usa una estrategia de transfer learning más avanzada, como fine-tuning o una combinación de bottleneck features y fine-tuning. Revisa las diapositivas de clase para saber cómo.
   
Si empleas todos estos trucos, es posible alcanzar más de un 90% de acierto en el conjunto de test.

</font>

***

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D

image_size = 224

train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=image_size, batch_size=64)

vgg16_model = VGG16(include_top=False, input_shape=(image_size, image_size, 3))
vgg16_model.trainable = False

custom_early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=8, 
    min_delta=0.001
)

bottleneck_model = Sequential()

bottleneck_model.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))
bottleneck_model.add(vgg16_model)


bottleneck_model.add(GlobalAveragePooling2D())
bottleneck_model.add(Dense(512, activation="relu"))
bottleneck_model.add(Dropout(0.5))
bottleneck_model.add(Dense(256, activation="relu"))
bottleneck_model.add(Dropout(0.5))
bottleneck_model.add(Dense(32, activation="relu"))
bottleneck_model.add(Dense(6, activation="softmax"))

bottleneck_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
bottleneck_model.fit(
    train_dataset,
    epochs=30,
    verbose=1 ,
    callbacks=[custom_early_stopping],
    validation_data=val_dataset
)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.
Epoch 1/30
96/96 [==============================] - 87s 836ms/step - loss: 2.1041 - accuracy: 0.3811 - val_loss: 1.0833 - val_accuracy: 0.6020
Epoch 2/30
96/96 [==============================] - 76s 777ms/step - loss: 1.1451 - accuracy: 0.5807 - val_loss: 0.8638 - val_accuracy: 0.6983
Epoch 3/30
96/96 [==============================] - 76s 778ms/step - loss: 0.9353 - accuracy: 0.6717 - val_loss: 0.7169 - val_accuracy: 0.7571
Epoch 4/30
96/96 [==============================] - 76s 780ms/step - loss: 0.8183 - accuracy: 0.7180 - val_loss: 0.6626 - val_accuracy: 0.7657
Epoch 5/30
96/96 [==============================] - 76s 776ms/step - loss: 0.7398 - accuracy: 0.7483 - val_loss: 0.6215 - val_accuracy: 0.7837
Epoch 6/30
96/96 [==============================] - 76s 777ms/step - loss: 0.6836 - accuracy: 0.7578 - val_loss: 0.5842 - val_accuracy: 0.8008
Epoch 7/30
96/96 [=

In [ ]:
bottleneck_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_1 (Lambda)           (None, 224, 224, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_9 (Dense)             (None, 512)               262656    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 256)               131328    
                                                      

In [ ]:
score = bottleneck_model.evaluate(test_dataset)
print("Test loss", score[0])
print("Test accuracy", score[1])

33/33 [==============================] - 22s 638ms/step - loss: 0.5124 - accuracy: 0.8556
Test loss 0.5124409198760986
Test accuracy 0.855555534362793


In [ ]:
image_size = 224

train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=image_size, batch_size=64)

vgg16_model = VGG16(include_top=False, input_shape=(image_size, image_size, 3))
vgg16_model.trainable = False

custom_early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=8, 
    min_delta=0.001
)

bottleneck_model_2 = Sequential([
              RandomFlip('horizontal'),
              RandomRotation(0.2),
              RandomZoom(0.3,0.2),
              BatchNormalization()
             ])

bottleneck_model_2.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))
bottleneck_model_2.add(vgg16_model)

bottleneck_model_2.add(Flatten())
bottleneck_model_2.add(Dense(512, activation="relu"))
bottleneck_model_2.add(Dropout(0.5))
bottleneck_model_2.add(Dense(256, activation="relu"))
bottleneck_model_2.add(Dropout(0.5))
bottleneck_model_2.add(Dense(32, activation="relu"))
bottleneck_model_2.add(Dense(6, activation="softmax"))

bottleneck_model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
bottleneck_model_2.fit(
    train_dataset,
    epochs=20,
    verbose=1 ,
    callbacks=[custom_early_stopping],
    validation_data=val_dataset
)

score = bottleneck_model_2.evaluate(test_dataset)
print("Test loss", score[0])
print("Test accuracy", score[1])

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.
Epoch 1/20
96/96 [==============================] - 135s 1s/step - loss: 2.1112 - accuracy: 0.1986 - val_loss: 1.7797 - val_accuracy: 0.2173
Epoch 2/20
96/96 [==============================] - 128s 1s/step - loss: 1.7944 - accuracy: 0.2213 - val_loss: 1.7685 - val_accuracy: 0.2130
Epoch 3/20
96/96 [==============================] - 127s 1s/step - loss: 1.7666 - accuracy: 0.2262 - val_loss: 1.7594 - val_accuracy: 0.2372
Epoch 4/20
96/96 [==============================] - 127s 1s/step - loss: 1.7568 - accuracy: 0.2438 - val_loss: 1.7538 - val_accuracy: 0.2372
Epoch 5/20
96/96 [==============================] - 128s 1s/step - loss: 1.7426 - accuracy: 0.2484 - val_loss: 1.7361 - val_accuracy: 0.2372
Epoch 6/20
96/96 [==============================] - 128s 1s/step - loss: 1.7395 - accuracy: 0.2440 - val_loss: 1.7306 - val_accuracy: 0.2372
Epoch 7/20
96/96 [=============

In [ ]:
from tensorflow.keras.applications import ResNet50

image_size = 224

train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=image_size, batch_size=64)

resNet_model = ResNet50(include_top=False, input_shape=(image_size, image_size, 3))
resNet_model.trainable = False

custom_early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=8, 
    min_delta=0.001
)

bottleneck_model_3 = Sequential()

bottleneck_model_3.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))
bottleneck_model_3.add(resNet_model)

bottleneck_model_3.add(GlobalAveragePooling2D())
bottleneck_model_3.add(Dense(512, activation="relu"))
bottleneck_model_3.add(Dropout(0.5))
bottleneck_model_3.add(Dense(256, activation="relu"))
bottleneck_model_3.add(Dropout(0.5))
bottleneck_model_3.add(Dense(32, activation="relu"))
bottleneck_model_3.add(Dense(6, activation="softmax"))

bottleneck_model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
bottleneck_model_3.fit(
    train_dataset,
    epochs=30,
    verbose=1 ,
    callbacks=[custom_early_stopping],
    validation_data=val_dataset
)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.
Epoch 1/30
96/96 [==============================] - 62s 580ms/step - loss: 1.2570 - accuracy: 0.5243 - val_loss: 0.6790 - val_accuracy: 0.7723
Epoch 2/30
96/96 [==============================] - 53s 544ms/step - loss: 0.7490 - accuracy: 0.7320 - val_loss: 0.5695 - val_accuracy: 0.8022
Epoch 3/30
96/96 [==============================] - 53s 545ms/step - loss: 0.6173 - accuracy: 0.7840 - val_loss: 0.4947 - val_accuracy: 0.8278
Epoch 4/30
96/96 [==============================] - 53s 546ms/step - loss: 0.5295 - accuracy: 0.8201 - val_loss: 0.4537 - val_accuracy: 0.8444
Epoch 5/30
96/96 [==============================] - 53s 543ms/step - loss: 0.4818 - accuracy: 0.8354 - val_loss: 0.4540 - val_accuracy: 0.8454
Epoch 6/30
96/96 [==============================] - 53s 544ms/step - loss: 0.4262 - accuracy: 0.8512 - val_loss: 0.4361 - val_accuracy: 0.8525
Epoch 7/30
96/96 [=

In [ ]:
bottleneck_model_3.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_3 (Lambda)           (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_17 (Dense)            (None, 512)               1049088   
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_18 (Dense)            (None, 256)               131328    
                                                      

In [ ]:
score = bottleneck_model_3.evaluate(test_dataset)
print("Test loss", score[0])
print("Test accuracy", score[1])

33/33 [==============================] - 15s 426ms/step - loss: 0.5844 - accuracy: 0.8618
Test loss 0.5844084620475769
Test accuracy 0.861835777759552


In [ ]:
from tensorflow.keras.applications import Xception

image_size = 299

train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=image_size, batch_size=64)

xception_model = Xception(include_top=False, input_shape=(image_size, image_size, 3))
xception_model.trainable = False

custom_early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=8, 
    min_delta=0.001
)

bottleneck_model_4 = Sequential()

bottleneck_model_4.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))
bottleneck_model_4.add(xception_model)

bottleneck_model_4.add(GlobalAveragePooling2D())
bottleneck_model_4.add(Dense(512, activation="relu"))
bottleneck_model_4.add(Dropout(0.5))
bottleneck_model_4.add(Dense(256, activation="relu"))
bottleneck_model_4.add(Dropout(0.5))
bottleneck_model_4.add(Dense(32, activation="relu"))
bottleneck_model_4.add(Dense(6, activation="softmax"))

bottleneck_model_4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
bottleneck_model_4.fit(
    train_dataset,
    epochs=30,
    verbose=1 ,
    callbacks=[custom_early_stopping],
    validation_data=val_dataset
)

Found 6082 files belonging to 6 classes.
Found 2108 files belonging to 6 classes.
Found 2070 files belonging to 6 classes.
83697664/83683744 [==============================] - 0s 0us/step
Epoch 1/30
96/96 [==============================] - 79s 658ms/step - loss: 2.2924 - accuracy: 0.2159 - val_loss: 1.7571 - val_accuracy: 0.2443
Epoch 2/30
96/96 [==============================] - 63s 646ms/step - loss: 1.7581 - accuracy: 0.2540 - val_loss: 1.7176 - val_accuracy: 0.2951
Epoch 3/30
96/96 [==============================] - 64s 658ms/step - loss: 1.7211 - accuracy: 0.2688 - val_loss: 1.6824 - val_accuracy: 0.3074
Epoch 4/30
96/96 [==============================] - 65s 664ms/step - loss: 1.7033 - accuracy: 0.3011 - val_loss: 1.6665 - val_accuracy: 0.3178
Epoch 5/30
96/96 [==============================] - 66s 676ms/step - loss: 1.6553 - accuracy: 0.3172 - val_loss: 1.6271 - val_accuracy: 0.3425
Epoch 6/30
96/96 [==============================] - 65s 673ms/step - loss: 1.6343 - accuracy: 0.3

In [ ]:
bottleneck_model_4.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 299, 299, 3)       0         
                                                                 
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                        

In [ ]:
score = bottleneck_model_4.evaluate(test_dataset)
print("Test loss", score[0])
print("Test accuracy", score[1])

33/33 [==============================] - 17s 489ms/step - loss: 1.4162 - accuracy: 0.4517
Test loss 1.4161601066589355
Test accuracy 0.45169082283973694


## Informe final y resumen de resultados

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Escribe en la siguiente celda un pequeño informe con:
    <ul>
        <li>Una tabla de resultados, indicando qué diseños de red has probado y qué resultados en test has obtenido. Puede usar un estilo de tabla como el que se muestra abajo.</li>
        <li>De las estrategias y diseños que has ido probando, ¿qué ha funcionado y qué no?</li>
        <li>¿Qué has aprendido con esta práctica?
    </ul>
</font>

***

Ejemplo para la tabla de resultados

|Procesado de imágenes|Diseño de red neuronal|Estrategia de entrenamiento|Acierto en test|
|---------------------|----------------------|---------------------------|---------------|
|Tamaño 128x128; batch size 64; augmentation: flip, rotation, zoom; batch normalitation |(Convolutional(32) + MaxPool(3x3)) * 3 + Flatten + Dense(256) + Drop (0.5) + Dense(32) + Dense(6) |Entrenamiento desde 0|56%|
|Tamaño 128x128; batch size 64; augmentation: flip, rotation, zoom; batch normalitation |Convolutional(64) + MaxPool(3x3) + (Convolutional(32) + MaxPool(3x3)) * 2 + Flatten + Dense(512) + Drop (0.5) + Dense(256) + Drop(0.5) + Dense(32) + Dense(6) |Entrenamiento desde 0|54%|
|Tamaño 224x224; batch size 64 |VGG16 + GlobalAveragePool + Dense(512) + Drop(0.5) + Dense(256) + Drop(0.5) + Dense(32) + Dense(6)|Bottleneck features|85%|
|Tamaño 224x224; batch size 64; augmentation: flip, rotation, zoom; batch normalitation |VGG16 + GlobalAveragePool + Dense(512) + Drop(0.5) + Dense(256) + Drop(0.5) + Dense(32) + Dense(6)|Bottleneck features|23%|
|Tamaño 224x224; batch size 64 |ResNet50 + GlobalAveragePool + Dense(512) + Drop(0.5) + Dense(256) + Drop(0.5) + Dense(32) + Dense(6)|Bottleneck features|87%|
|Tamaño 299x299; batch size 64 |Xception + GlobalAveragePool + Dense(512) + Drop(0.5) + Dense(256) + Drop(0.5) + Dense(32) + Dense(6)|Bottleneck features|45%|


El modelo con mejor accuracy es el modelo de **transfer learning** con **ResNet50**, con un **87% accuracy**. El peor modelo, por otro lado, es el model **VGG-16**, con augmentation de imágenes, el cual se queda en un **23%** de accuracy.

En los dos primeros modelos hemos usado 3 kernels, pero al aumentar su número a 5, hemos obtenido reusltados peores. 

El proceso de **augmentation de imágenes**, nos **ayudó** mucho en las **redes convulucionales iniciales**, pero con el **VGG-16 empeoraba** muchísimo el rendimineto del modelo.

Hemos aprendido, que no siempre es mejor inventar, si no que se obtienen mejores resultados aprovechando el trabajo realizado por otros e intentando mejorarlo.